In [2]:
%defaultDatasource jdbc:h2:mem:db
DROP Table IF EXISTS ParticipacaoComite;
DROP Table IF EXISTS Atleta;
DROP Table IF EXISTS EsporteModalidade;
DROP TABLE IF EXISTS EdicaoDosJogos;
DROP TABLE IF EXISTS ComiteOlimpico;
DROP TABLE IF EXISTS ParticipacaoAtletas;
DROP TABLE IF EXISTS ComiteDosAtletas;
DROP TABLE IF EXISTS EsportesDasEdicoes;

In [3]:
CREATE TABLE EdicaoDosJogos (
    Ano INTEGER NOT NULL,
    NumeroDaEdicao INTEGER NOT NULL,
    CidadeSede VARCHAR(20) NOT NULL,
    TotalDeAtletas INTEGER NOT NULL,
    Mascote VARCHAR(50) NOT NULL,
    PRIMARY KEY (Ano)
) AS SELECT
    Ano,
    NumeroDaEdicao,
    Cidade,
    TotalDeAtletas,
    Mascote
FROM CSVREAD('./edicoes.csv');

In [4]:
CREATE TABLE Atleta (
    Id INTEGER NOT NULL,
    Nome VARCHAR(100) NOT NULL,
    AnoDeNascimento VARCHAR(10),
    Sexo VARCHAR(1),
    PRIMARY KEY (Id)
) AS SELECT
    Id,
    Nome,
    AnoDeNascimento,
    Sexo
FROM CSVREAD('./atletas.csv');

In [5]:
CREATE TABLE ComiteOlimpico (
    Sigla VARCHAR(3) NOT NULL,
    Pais VARCHAR(50) NOT NULL,
    PRIMARY KEY (Sigla)
) AS SELECT
    Sigla,
    País
FROM CSVREAD('./IOC.csv');

In [6]:
CREATE TABLE EsporteModalidade (
    Id INTEGER NOT NULL,
    Nome VARCHAR(100) NOT NULL,
    EsportePai VARCHAR(100),
    PRIMARY KEY (Id)
) AS SELECT
    Id,
    Nome,
    EsportePai
FROM CSVREAD('./esportes.csv');

In [7]:
CREATE TABLE ParticipacaoComites  (
    IdComite VARCHAR(3) NOT NULL,
    AnoEdicao INTEGER NOT NULL,
    QtdAtletas INTEGER NOT NULL,
    QtdOuro INTEGER NOT NULL,
    QtdPrata INTEGER NOT NULL,
    QtdBronze INTEGER NOT NULL,
    Classificacao VARCHAR(3) NOT NULL,
    FOREIGN KEY (IdComite)
        REFERENCES ComiteOlimpico(Sigla)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (AnoEdicao)
        REFERENCES EdicaoDosJogos(Ano)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT
    IdComite,
    AnoEdicao,
    QtdAtletas,
    QtdOuro,
    QtdPrata,
    QtdBronze,
    Classificacao
FROM CSVREAD('./participacaoComites.csv')

In [ ]:
CREATE TABLE ParticipacaoAtletas  (
    IdAtleta INTEGER NOT NULL,
    AnoEdicao INTEGER NOT NULL,
    IdModalidade INTEGER NOT NULL,
    Altura VARCHAR(16) NOT NULL,
    Peso VARCHAR(16) NOT NULL,
    Medalha VARCHAR(6) NOT NULL,
    FOREIGN KEY (IdAtleta)
        REFERENCES Atleta(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (AnoEdicao)
        REFERENCES EdicaoDosJogos(Ano)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (IdModalidade)
        REFERENCES EsporteModalidade(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT
    IdAtleta,
    AnoEdicao,
    IdModalidade,
    Altura,
    Peso,
    Medalha
FROM CSVREAD('./participacaoAtletas.csv')

In [ ]:
CREATE TABLE ComiteDosAtletas  (
    IdComite VARCHAR(3) NOT NULL,
    IdAtleta INTEGER NOT NULL,
    FOREIGN KEY (IdComite)
        REFERENCES ComiteOlimpico(Sigla)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (IdAtleta)
        REFERENCES Atleta(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT
    IdComite,
    IdAtleta
FROM CSVREAD('./comiteDosAtletas.csv')

In [ ]:
CREATE TABLE EsportesDasEdicoes  (
    AnoEdicao INTEGER NOT NULL,
    IdModalidade INTEGER NOT NULL,
    Ouro VARCHAR(3),
    Prata VARCHAR(3),
    Bronze VARCHAR(3),
    FOREIGN KEY (AnoEdicao)
        REFERENCES EdicaoDosJogos(Ano)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (IdModalidade)
        REFERENCES EsporteModalidade(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION
) AS SELECT
    AnoEdicao,
    IdModalidade,
    Ouro,
    Prata,
    Bronze
FROM CSVREAD('./esportesDasEdicoes.csv')

In [ ]:
SELECT * FROM EDICAODOSJOGOS

In [ ]:
/*Pergunta 1*/
SELECT DISTINCT C.PAIS, CAST (P.Classificacao AS INT) Classificacao
    FROM ParticipacaoComites P, ComiteOlimpico C
    WHERE C.Sigla=P.IdComite AND AnoEdicao=2016 AND Classificacao<>'-'
    ORDER BY Classificacao

In [ ]:
/*Pergunta 2*/
SELECT C.PAIS, (SUM(P.QTDOURO)+SUM(P.QTDPRATA)+SUM(P.QTDBRONZE))/COUNT(*) MEDIA
    FROM PARTICIPACAOCOMITES P, COMITEOLIMPICO C
    WHERE P.IDCOMITE=C.SIGLA
    GROUP BY C.PAIS
    ORDER BY MEDIA DESC

In [ ]:
/*Pergunta 3*/
SELECT DISTINCT E.ANOEDICAO, M.NOME, E.OURO
    FROM ESPORTESDASEDICOES E, ESPORTEMODALIDADE M
    WHERE E.IDMODALIDADE=M.ID AND E.ANOEDICAO >= 2000 AND M.ID=8
    ORDER BY E.ANOEDICAO

In [ ]:
/*Pergunta 4*/
SELECT P.ANOEDICAO, A.SEXO, COUNT(*) TOTAL
    FROM ATLETA A, PARTICIPACAOATLETAS P
    WHERE A.ID=P.IDATLETA
    GROUP BY P.ANOEDICAO, A.SEXO
    ORDER BY P.ANOEDICAO

In [35]:
/*Pergunta 5*/
SELECT A.NOME, P.MEDALHA, COUNT(*) TOTAL
    FROM PARTICIPACAOATLETAS P, ATLETA A
    WHERE P.IDATLETA=A.ID AND A.ID=1467 AND P.MEDALHA<>'-'
    GROUP BY P.IDATLETA, P.MEDALHA

In [34]:
SELECT NOME, COUNT(*) TOTALOLIMPIADAS
    FROM (SELECT A.NOME, COUNT(*) TOTALJOGOS
            FROM PARTICIPACAOATLETAS P, ATLETA A
            WHERE P.IDATLETA=A.ID AND A.ID=1467
            GROUP BY P.IDATLETA, P.ANOEDICAO)
    GROUP BY NOME